##### STA 141B Data & Web Technologies for Data Analysis

# Lecture 13 - 02/19/26, ISS + Selenium

### Announcements
- Midterm solutions will be discussed in the discussion sections on February 18.

### Today's topics
 - Selenium Browser

### Ressources
 - [WhereTheISS](wheretheiss.at)

## ISS and Satellite Data

Let's have a look on this great website:
https://wheretheiss.at/

It even provides a documented API! Find the documentation [here](https://wheretheiss.at/w/developer)!

#### Introduction

In [1]:
import requests
import requests_cache
import pandas as pd

headers = {
    'User - agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:142.0) Gecko/20100101 Firefox/142.0',
}

session = requests_cache.CachedSession('../output/ISS3')

The documentation says it contains a lot of satellites plus the ISS.

So, let's first check all available satellites!

In [2]:
url = 'https://api.wheretheiss.at/v1/satellites'

response = session.get(url, headers = headers)
response.raise_for_status()
print(response.headers)

{'Date': 'Mon, 16 Feb 2026 20:40:21 GMT', 'Server': 'Apache/2.2.22 (Ubuntu)', 'X-Powered-By': 'PHP/5.3.10-1ubuntu3.26', 'X-Rate-Limit-Limit': '350', 'X-Rate-Limit-Remaining': '349', 'X-Rate-Limit-Interval': '5 minutes', 'Access-Control-Allow-Origin': '*', 'X-Apache-Time': 'D=20085', 'Cache-Control': 'max-age=0, no-cache', 'Content-Length': '27', 'Keep-Alive': 'timeout=15, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'application/json'}


In [3]:
pd.DataFrame(list(response.headers.items()))

,0,1
0,Date,"Mon, 16 Feb 2026 20:40:21 GMT"
1,Server,Apache/2.2.22 (Ubuntu)
2,X-Powered-By,PHP/5.3.10-1ubuntu3.26
3,X-Rate-Limit-Limit,350
4,X-Rate-Limit-Remaining,349
5,X-Rate-Limit-Interval,5 minutes
6,Access-Control-Allow-Origin,*
7,X-Apache-Time,D=20085
8,Cache-Control,"max-age=0, no-cache"
9,Content-Length,27


We can get the rate limits from the headers!

Now, lets check out all satellites:

In [4]:
all_satelites = print(response.json())

[{'name': 'iss', 'id': 25544}]


Only the ISS :(

In [5]:
iss_id = str(response.json()[0]['id'])

Get the location of the ISS:

In [6]:
import requests

url = 'https://api.wheretheiss.at/v1/satellites/' + iss_id

response = session.get(url, headers = headers)
response.raise_for_status()
print(response.headers)

{'Date': 'Mon, 16 Feb 2026 20:40:21 GMT', 'Server': 'Apache/2.2.22 (Ubuntu)', 'X-Powered-By': 'PHP/5.3.10-1ubuntu3.26', 'X-Rate-Limit-Limit': '350', 'X-Rate-Limit-Remaining': '348', 'X-Rate-Limit-Interval': '5 minutes', 'Access-Control-Allow-Origin': '*', 'X-Apache-Time': 'D=19539', 'Cache-Control': 'max-age=0, no-cache', 'Content-Length': '310', 'Keep-Alive': 'timeout=15, max=99', 'Connection': 'Keep-Alive', 'Content-Type': 'application/json'}


In [7]:
lim_remaining = response.headers["X-Rate-Limit-Remaining"]
print(f'Remaining requests within the next five minutes: {lim_remaining}')

Remaining requests within the next five minutes: 348


In [8]:
print(response) # shows the status code, not the content itself

<Response [200]>


Note: this is a cached response!

In [9]:
data = response.json()
print(data) # here we go!

{'name': 'iss', 'id': 25544, 'latitude': 5.0206459875982, 'longitude': 82.337004225013, 'altitude': 422.1139070127, 'velocity': 27572.431926443, 'visibility': 'eclipsed', 'footprint': 4518.2024726005, 'timestamp': 1771274421, 'daynum': 2461088.3613542, 'solar_lat': -12.104317805665, 'solar_lon': 233.4141431277, 'units': 'kilometers'}


In [10]:
import pandas as pd

tbl = pd.DataFrame(data, index=[0])

In [11]:
tbl.head() # looks good

,name,id,latitude,longitude,altitude,velocity,visibility,footprint,timestamp,daynum,solar_lat,solar_lon,units
0,iss,25544,5.020646,82.337004,422.113907,27572.431926,eclipsed,4518.202473,1771274421,2.461088e+06,-12.104318,233.414143,kilometers


<p>Find the developer rate limits <a href="https://wheretheiss.at/w/developer">here</a>!</p>

<div class="alert alert-danger">
    We MUST NOT send more then one request per second!
</div>


#### Accessing the location

Let's access the ISS' location for a specific timestamp:

In [12]:
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timezone, timedelta
import warnings
import time

In [13]:
def get_positions(timestamps):
    response = session.get('https://api.wheretheiss.at/v1/satellites/' + iss_id + '/positions?timestamps=' + str(timestamps))
    response.raise_for_status()
    lim_remaining = response.headers["X-Rate-Limit-Remaining"]
    if int(lim_remaining) < 10: # only slow down if we have indeed 
        warnings.warn("Too many requests!")
    time.sleep(1)
    return(response.json())

def is_inlighted(data):
    status = data["visibility"]
    return(status == "daylight")

#### Insertion: timedate formats
Dealing with different timedate formats

In [14]:
tz = 'America/Los_Angeles'
def dt2epoch(date):
    tme = np.datetime64(date)
    return(tme.astype(np.int64))

print(dt2epoch("now"))
print(dt2epoch("2025-02-05 15:00:00"))
print(dt2epoch("2025-02-05 15:00:00-0800"))

1771274425
1738767600
1738796400


/var/folders/qz/ztp_k9054wz3wyfvjqbw10_h0000gp/T/ipykernel_4756/3763540327.py:3: UserWarning: no explicit representation of timezones available for np.datetime64
  tme = np.datetime64(date)


In [15]:
tz = 'America/Los_Angeles'
def epoch2dt(epoch):
    return(pd.to_datetime(epoch, unit = "s", utc=True).tz_convert(tz))

In [16]:
current_time = np.datetime64('now') # np.datetime64('2026-02-05 05:00:00-0800') # np.datetime64('now')

In [17]:
current_time

np.datetime64('2026-02-16T20:40:25')

In [18]:
dt2epoch(current_time)

np.int64(1771274425)

In [19]:
epoch2dt(dt2epoch(current_time))

Timestamp('2026-02-16 12:40:25-0800', tz='America/Los_Angeles')

#### DUSK/DAWN

We consider twilight to be everything 30 minutes before and after dawn or dusk. (This is a very rough proxy!)

https://www.timeanddate.com/sun/usa/davis

In [20]:
current_time

np.datetime64('2026-02-16T20:40:25')

In [21]:
!pip install astral

In [22]:
from astral import LocationInfo
from astral.sun import sun
import datetime as dt

tz = 'America/Los_Angeles'
timestamp_dt = epoch2dt(current_time)
davis = LocationInfo("Davis", "California", tz, 38.544907, -121.740517)
s = sun(davis.observer, date=timestamp_dt.date(), tzinfo=davis.timezone)
sun_times = pd.Series([s['dawn'], s['dusk']])

In [23]:
sun_times

0   2026-02-16 06:28:50.968572-08:00
1   2026-02-16 18:13:41.763421-08:00
dtype: datetime64[ns, America/Los_Angeles]

In [24]:
from astral import LocationInfo
from astral.sun import sun
import datetime as dt
import pandas as pd

tz = 'America/Los_Angeles'
davis = LocationInfo("Davis", "California", tz, 38.544907, -121.740517)

today = pd.Timestamp("02-10-2026") # pd.Timestamp("today")
print('Current time: ' + str(today))

twilight = []

for i in range(4):
    day = today + pd.Timedelta(i, "days")
    s = sun(davis.observer, date=day, tzinfo=davis.timezone)

    twilight += [(s['dawn'] - pd.Timedelta(75, "minutes") + pd.Timedelta(sec, "seconds")).timestamp() for sec in range(0,90*60,10)]
    twilight += [(s['dusk'] - pd.Timedelta(15, "minutes") + pd.Timedelta(sec, "seconds")).timestamp() for sec in range(0,90*60,10)]
    print('Dawn for today + ' + str(i) + " day(s): " + str(s['dawn']))
    print('Dusk for today + ' + str(i) + " day(s): " + str(s['dusk']))

Current time: 2026-02-10 00:00:00
Dawn for today + 0 day(s): 2026-02-10 06:35:34.361979-08:00
Dusk for today + 0 day(s): 2026-02-10 18:07:18.645880-08:00
Dawn for today + 1 day(s): 2026-02-11 06:34:30.466978-08:00
Dusk for today + 1 day(s): 2026-02-11 18:08:22.953305-08:00
Dawn for today + 2 day(s): 2026-02-12 06:33:25.195902-08:00
Dusk for today + 2 day(s): 2026-02-12 18:09:27.091324-08:00
Dawn for today + 3 day(s): 2026-02-13 06:32:18.579461-08:00
Dusk for today + 3 day(s): 2026-02-13 18:10:31.049700-08:00


In [25]:
print(twilight[0:10])
print(len(twilight))

[1770729634.361979, 1770729644.361979, 1770729654.361979, 1770729664.361979, 1770729674.361979, 1770729684.361979, 1770729694.361979, 1770729704.361979, 1770729714.361979, 1770729724.361979]
4320


In [26]:
twilight

[1770729634.361979,
 1770729644.361979,
 1770729654.361979,
 1770729664.361979,
 1770729674.361979,
 1770729684.361979,
 1770729694.361979,
 1770729704.361979,
 1770729714.361979,
 1770729724.361979,
 1770729734.361979,
 1770729744.361979,
 1770729754.361979,
 1770729764.361979,
 1770729774.361979,
 1770729784.361979,
 1770729794.361979,
 1770729804.361979,
 1770729814.361979,
 1770729824.361979,
 1770729834.361979,
 1770729844.361979,
 1770729854.361979,
 1770729864.361979,
 1770729874.361979,
 1770729884.361979,
 1770729894.361979,
 1770729904.361979,
 1770729914.361979,
 1770729924.361979,
 1770729934.361979,
 1770729944.361979,
 1770729954.361979,
 1770729964.361979,
 1770729974.361979,
 1770729984.361979,
 1770729994.361979,
 1770730004.361979,
 1770730014.361979,
 1770730024.361979,
 1770730034.361979,
 1770730044.361979,
 1770730054.361979,
 1770730064.361979,
 1770730074.361979,
 1770730084.361979,
 1770730094.361979,
 1770730104.361979,
 1770730114.361979,
 1770730124.361979,


#### GET DATA

In [ ]:
import tqdm, requests
pos = []

url = "https://api.wheretheiss.at/v1/satellites/25544/positions?timestamps="

def get_positions(timestamps):
    response = session.get(url + str(timestamps))
    response.raise_for_status()
    lim_remaining = response.headers["X-Rate-Limit-Remaining"]
    if int(lim_remaining) < 200:
        warnings.warn("Two many requests!")
        time.sleep(1)
    time.sleep(1)
    return(response.json())

for ind in tqdm.tqdm(range(0, len(twilight), 10)):
#    progress = np.round(100*ind/len(epoch_time),1)
    timestamps = twilight[ind:(ind+10)]
    ts_string = ",".join(map(str, timestamps))
    
    pos += get_positions(ts_string)
    # time.sleep(1)

print(pos)

  0%|          | 2/432 [00:02<08:00,  1.12s/it]

##### SAVE / LOAD DATA

In [ ]:
import numpy as np
import pandas as pd
import requests
import requests_cache

In [ ]:
import json

with open('../output/ISS2.json','w+') as file:
    json.dump(pos, file)

In [ ]:
import json

with open('../output/ISS.json','r') as file:
    pos = json.load(file)

##### CONTINUE

In [ ]:
data = pd.DataFrame(pos)
data = data[data.visibility == "daylight"]
perc = int(len(data)/len(pos)*100)

print(f'The ISS is in daylight in {perc}% of the cases.')
data = data.drop(['name', 'id', 'velocity', 'visibility', 'daynum', 'solar_lat', 'solar_lon', 'units'], axis = 1).set_index("timestamp")
print(data)

In [ ]:
!pip install geopy

In [ ]:
from geopy.distance import geodesic

davis = (38.544907, -121.740517)

def calc_diff(coords):
    point = (coords["latitude"], coords["longitude"])
    return(geodesic(point, davis).km)

dist = data.apply(calc_diff, axis=1)
data["distance"] = dist
near = (dist < data["footprint"])
data["is_near"] = near
data.head()

In [ ]:
tbl = data[data["is_near"]]
len(tbl)

In [ ]:
tbl

##### SKIP THIS

In [ ]:
from astral import LocationInfo
from astral.sun import sun
import datetime as dt

def is_twilight(entry):
    tz = 'America/Los_Angeles'
    timestamp_dt = epoch2dt(entry["timestamp"]) # convert timestamp to epoch time
    davis = LocationInfo("Davis", "California", tz, 38.544907, -121.740517)
    s = sun(davis.observer, date=timestamp_dt.date(), tzinfo=davis.timezone) # providing the data of the timestamp
    sun_times = pd.Series([s['dawn'], s['dusk']]) # calculating the timestamp fo the dawn and dusk on this specific day
#    time_diffs = np.abs(timestamp_dt - sun_times)
#    min_diff = time_diffs.min()
    if (timestamp_dt > s['dawn'] - pd.Timedelta(minutes=90) and timestamp_dt < s['dawn'] + pd.Timedelta(minutes=20)) or (
            timestamp_dt > s['dusk'] - pd.Timedelta(minutes=20) and timestamp_dt < s['dusk'] + pd.Timedelta(minutes=90)):
        tw = True
    else:
        tw = False
    return tw
#    return min_diff < pd.Timedelta(minutes=70)

twl = data.reset_index().apply(is_twilight, axis=1)
sum(twl)

In [ ]:
data.shape

In [ ]:
twl

Next, we should calculate the distance between Davis and the projection of the ISS on the earth.

Remember: we don't have to do everything by hand. For most applications, there is a Python package.

In [ ]:
!pip install geopy

In [ ]:
from geopy.distance import geodesic

davis = (38.544907, -121.740517)

def calc_diff(coords):
    point = (coords["latitude"], coords["longitude"])
    return(geodesic(point, davis).km)

dist = tbl.apply(calc_diff, axis=1)
dist.head()

##### CONTINUE HERE

In [ ]:
tbl.head()

In [ ]:
amin = np.argmin(tbl[tbl.is_near]["distance"])
minimizer = tbl[tbl.is_near].iloc[amin]
print(minimizer)

In [ ]:
epoch2dt(minimizer.name)

Compare it with: https://www.astroviewer.net/iss/en/observation.php

#### BEARING

In [ ]:
tz = 'America/Los_Angeles'
davis = [38.544907, -121.740517]

import math

def calculate_bearing(point):
    """
    Calculates the bearing in degrees between two geolocations.
    Latitudes and longitudes should be provided in decimal degrees.
    """
    lat1_rad = math.radians(davis[0])
    lon1_rad = math.radians(davis[1])
    lat2_rad = math.radians(point[0])
    lon2_rad = math.radians(point[1])

    delta_lon = lon2_rad - lon1_rad

    y = math.sin(delta_lon) * math.cos(lat2_rad)
    x = math.cos(lat1_rad) * math.sin(lat2_rad) - \
        math.sin(lat1_rad) * math.cos(lat2_rad) * math.cos(delta_lon)

    bearing_rad = math.atan2(y, x)
    bearing_deg = math.degrees(bearing_rad)
    
    # Normalize bearing to be within 0-360 degrees
    bearing_deg = (bearing_deg + 360) % 360
    return bearing_deg

def get_cardinal_direction(point):
    """
    Converts a bearing in degrees (0-360) to a cardinal direction.
    """
    directions = ["N", "NE", "E", "SE", "S", "SW", "W", "NW"]
    # Adjust bearing to align with the center of each 45-degree sector
    # For example, N is centered at 0, NE at 45, E at 90, etc.
    bearing = calculate_bearing(point)
    index = round(bearing / 45) % 8
    return directions[index]

In [ ]:
# Example Usage:
point = [38.8977, -77.0365] 

bearing = calculate_bearing(point)
direction = get_cardinal_direction(point)

print(f"The bearing from is {bearing:.2f} degrees.")
print(f"The cardinal direction is: {direction}")

The example point was the White House. It is indeed eastern of Davis, see:
[Google Maps](https://www.google.com/maps/dir/University+of+California,+Shields+Avenue,+Davis,+Kalifornien/38.8977,-77.0365/@38.2386811,-109.987335,5z/data=!3m1!4b1!4m9!4m8!1m5!1m1!1s0x80ead37f7489fa3f:0xecbfbb24087e8334!2m2!1d-121.7617125!2d38.5382322!1m0!3e4?entry=ttu&g_ep=EgoyMDI1MDkxNy4wIKXMDSoASAFQAw%3D%3D)

In [ ]:
point = list(minimizer[["latitude", "longitude"]])
print(calculate_bearing(point))
print(get_cardinal_direction(point))

Compare it with: https://www.astroviewer.net/iss/en/observation.php!

Waaaaaiiiit. Couldn't we just scrape this site?

In [ ]:
import requests
import requests_cache
import pandas as pd
import lxml.html as lx

In [ ]:
url = 'https://www.astroviewer.net/iss/en/observation.php'
headers = {
    'User - agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:142.0) Gecko/20100101 Firefox/142.0',
}

In [ ]:
response = requests.get(url)
response.raise_for_status()

In [ ]:
html = lx.fromstring(response.text)

In [ ]:
table = html.xpath('//table[contains(@class, "passDetails")]')[0]

In [ ]:
rows = table.xpath('//tbody/tr')

In [ ]:
cells = table.xpath('//tbody/tr/td')

In [ ]:
[c.text for c in cells]

In [ ]:
[c.text_content() for c in rows[0].xpath('//td')]

It conains only '...'. Have a look at the HTML content!

And how to choose Davis in the first place?

## --> SELENIUM BROWSER

# Selenium WebDriver

## Preparations

Before diving into Selenium’s features, let’s **install Chrome**, **configure ChromeDriver**, and install the Python **selenium** package.

Alternatively, you may also use the geckodriver for Firefox instead. See [here](https://www.selenium.dev/documentation/webdriver/browsers/firefox/) for more details about using Firefox through the geckodriver.

### Install the Selenium Library

In [ ]:
!pip install selenium

### Install the Browser Driver

There are two ways to set up a browser driver for Chrome:

1. **Manual Installation**  
   - Check your local **Chrome** version by typing `chrome://version` in Chrome’s address bar or via “Help → About Google Chrome.”  
   - Download the matching **ChromeDriver** from  
     <https://chromedriver.storage.googleapis.com/index.html>  
   - Either add the `chromedriver.exe` to your system’s PATH (e.g., drop it into Python’s `Scripts/` folder) or specify the absolute path directly in your code.

2. **Automatic Installation**  
   - Use a 3rd-party library such as **webdriver_manager** to install the appropriate driver automatically:

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

ChromeDriverManager().install() # detects your Chrome version, downloads the matching driver, and places it in your local cache.

With this setup in place, we can start using Selenium.

## Basic Usage

This section covers **initializing the browser**, visiting pages, setting the **browser window size**, **refreshing**, **forward/back** navigation, etc.

### Initialize a Browser Object

In [ ]:
# Option A: Direct initialization if ChromeDriver is in PATH
driver = webdriver.Chrome()

# Option B: Specify the absolute path to chromedriver
# path = r'C:\path\to\chromedriver.exe' for Windows
# driver = webdriver.Chrome(path)

driver.close()  # Closes the browser

### Access a Page

In [ ]:
import time

url = 'https://statistics.ucdavis.edu/'

with webdriver.Chrome() as driver:
    driver.get(url)
    time.sleep(3)
# driver.close() automatically closes the window afterwards

### Headless Browser

Having a browser doing things might be distracting. Let's use the headless mode!

In [ ]:
option = webdriver.ChromeOptions()
option.add_argument("headless") # no browser window visible

with webdriver.Chrome(options=option) as driver:
    driver.get(url)
    time.sleep(3)

### Screenshot

While using the headless mode may be useful in practice, you may take a screenshot sometimes, e.g., if an error occurs:

In [ ]:
with webdriver.Chrome(options=option) as driver:
    driver.get(url)
    driver.get_screenshot_as_file('../output/screenshot_ucd.png')

![Screenshot](../output/screenshot_ucd.png)

Well, that's only one quarter of the page. Seems like the browser windows is quite small, eh?

### Window Size

In [ ]:
with webdriver.Chrome() as driver:
    driver.maximize_window()          # Fullscreen
    driver.get(url)
#    driver.get_screenshot_as_file('../output/screenshot_ucd_max.png')
    time.sleep(2)

    driver.set_window_size(500, 500)  # 500 x 500
    time.sleep(2)

    driver.set_window_size(1000, 800) # 1000 x 800
    driver.get_screenshot_as_file('../output/screenshot_ucd_large.png')
    time.sleep(2)

![Screenshot](../output/screenshot_ucd_large.png)

### Page refresh

In [ ]:
url_dynamic = 'https://the-internet.herokuapp.com/dynamic_content'

with webdriver.Chrome() as driver:
    driver.maximize_window()          # Fullscreen
    driver.get(url_dynamic)
    time.sleep(2)
    driver.refresh()
    print('Page refreshed.')
    time.sleep(2)

### Forward/Back Navigation

In [ ]:
with webdriver.Chrome() as driver:
    driver.get(url_dynamic)
    time.sleep(2)
    driver.get(url)
    driver.back() # go back to the-internet
    time.sleep(2)
    driver.forward() # go forward to ucd
    time.sleep(2)

## Page Properties
Once Selenium opens a page, you can retrieve basic info:

In [ ]:
with webdriver.Chrome() as driver:
    driver.get(url)
    print(driver.title)       # page title
    print(driver.current_url) # current URL
    print(driver.name)        # browser name
    html = driver.page_source # raw HTML source

In [ ]:
html[:100]

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html)
links = soup.find_all('link')

In [ ]:
for link in links:
    print(link.get('href'))

## Page Elements

When using Selenium, a **key** step is to locate elements for input, clicking, etc. Below are common methods.

### Locating Page Elements

The syntax will always be 

```python
driver.find_element(By.X, "your_element_id")
```

where X is either an ID, Tag Name, etc

#### Locate by ID

In [ ]:
from selenium.webdriver.common.by import By
url_ab = 'https://the-internet.herokuapp.com/abtest'

with webdriver.Chrome() as driver:
    driver.get(url_ab)
    content = driver.find_element(By.ID, 'content')
    time.sleep(3)
    text = content.text

In [ ]:
print(text)

In [ ]:
url_test = 'https://automationintesting.com/selenium/testpage/'

with webdriver.Chrome() as driver:
    driver.get(url_test)
    driver.maximize_window()          # Fullscreen
    element = driver.find_element(By.ID, 'firstname')
    driver.execute_script("arguments[0].scrollIntoView();", element) # scroll until we can see the element
    time.sleep(2)
    element.send_keys('Aggies')
    time.sleep(5)

#### Locate by Name

In [ ]:
url_test = 'https://automationintesting.com/selenium/testpage/'

with webdriver.Chrome() as driver:
    driver.get(url_test)
    driver.maximize_window()          # Fullscreen
    element = driver.find_element(By.NAME, 'colour')
    driver.execute_script("window.scrollBy(0, 500);")  # scroll down by 500 pixels
    time.sleep(2)
    element.click()
    time.sleep(2)

#### Locate by Class Name

In [ ]:
url_test = 'https://automationintesting.com/selenium/testpage/'

with webdriver.Chrome() as driver:
    driver.get(url_test)
    element = driver.find_element(By.CLASS_NAME, 'info-title')
    title = element.text
    time.sleep(2)

print(title)

#### Locate by Tag Name

```python
browser.find_element(By.ID, 'name')
browser.find_element(By.NAME, 'name')
browser.find_element(By.CLASS_NAME, 'name')
browser.find_element(By.TAG_NAME, 'name')
browser.find_element(By.LINK_TEXT, 'name')
browser.find_element(By.PARTIAL_LINK_TEXT, 'name')
browser.find_element(By.XPATH, '//*[@id="name"]')
browser.find_element(By.CSS_SELECTOR, '#name')
```

Note that finding elements by using commands like
```browser.find_element_by_css_selector('#kw')```
are deprecated. It is highly recommended to use the `By.CSS_SELECTOR` instead.

In [ ]:
url_test = 'https://automationintesting.com/selenium/testpage/'

with webdriver.Chrome() as driver:
    driver.get(url_test)
    element = driver.find_element_by_name('info-title')
    title = element.text
    time.sleep(2)

print(title)

### Multiple Elements

If there are multiple matches, use `find_elements_...()` to get a **list** of matching elements.

## 4. Getting Element Attributes

### `get_attribute()`

For example, retrieving the `src` of an `<img>` element:

In [ ]:
url = 'https://statistics.ucdavis.edu/'

with webdriver.Chrome() as driver:
    driver.get(url)
    time.sleep(1)
    element = driver.find_element(By.XPATH, '//*[@id="block-hbwelcometotheucdavisdepartmentofstatistics"]/div/img')
    img_src = element.get_attribute('src')
    time.sleep(3)z

print(img_src)

### Getting Text

In [ ]:
url = 'https://statistics.ucdavis.edu/'

with webdriver.Chrome() as driver:
    driver.get(url)
    time.sleep(1)
    elements = driver.find_elements(By.XPATH, '//a')
    for el in elements:
        if el.text:
            print(el.text + ": " + el.get_attribute('href'))
    time.sleep(3)

#### Other Attributes

url = 'https://statistics.ucdavis.edu/'

with webdriver.Chrome() as driver:
    driver.get(url)
    time.sleep(1)
    element = driver.find_element(By.XPATH, '//*[@id="block-hbwelcometotheucdavisdepartmentofstatistics"]/div/img')
    img_src = element.get_attribute('src')
    time.sleep(3)

print(img_src)

print(logo.id)
print(logo.location)
print(logo.tag_name)
print(logo.size)

In [ ]:
url = 'https://statistics.ucdavis.edu/'

with webdriver.Chrome() as driver:
    driver.get(url)
    time.sleep(1)
    logo = driver.find_element(By.XPATH, '//*[@id="block-hbwelcometotheucdavisdepartmentofstatistics"]/div/img')

    print(logo.id)
    print(logo.location)
    print(logo.tag_name)
    print(logo.size)

    time.sleep(3)

### Page Interaction

We have already seen some interactions: 
- scrolling
- button clicks
- writing text

In [ ]:
url_test = 'https://automationintesting.com/selenium/testpage/'

driver = webdriver.Chrome()
driver.get(url_test)

In [ ]:
element = driver.find_element(By.ID, 'firstname')
driver.execute_script("arguments[0].scrollIntoView();", element)

In [ ]:
element.send_keys('Aggies') # write text

In [ ]:
element.clear() # clear field

In [ ]:
element.send_keys('NewAggies')

In [ ]:
driver.find_element(By.ID, 'submitbutton').click() # press button

In [ ]:
driver.find_element(By.ID, 'submitbutton').submit() # press enter

In [ ]:
driver.find_element(By.ID, 'gender').click()
driver.find_element(By.XPATH, "//option[@value='my_business']").click()
element = driver.find_element(By.ID, 'firstname').send_keys('Aggies')

In [ ]:
continents = driver.find_element(By.ID, "continent")

# Examples of what you can do:
print(option_element.text)          # Get the visible text
print(option_element.is_selected()) # Check if it is currently picked

In [ ]:
driver.quit()

## Delayed Waiting

Sometimes elements load dynamically. We have:

1. **`time.sleep(n)`** – forcibly pause n seconds.
2. **Implicit Wait**: `browser.implicitly_wait(10)`
3. **Explicit Wait**: with `WebDriverWait`.

In [ ]:
import time

driver = webdriver.Chrome()
driver.get('https://www.astroviewer.net/iss/en/observation.php')

time.sleep(2)
input_field = driver.find_element(By.ID, "locSearch") 
input_field.send_keys("Davis, CA")
input_field.send_keys(Keys.ENTER)
time.sleep(2)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

browser = webdriver.Chrome()
browser.get('https://www.astroviewer.net/iss/en/observation.php')

results_div = browser.find_element(By.ID, "passesHeader")
old_text = results_div.text
print(old_text)

lc_box = browser.find_element(By.ID, 'locSearch')
lc_box.send_keys('Davis, CA')
lc_box.send_keys(Keys.ENTER)

# Wait for the text to change (Custom Lambda)
WebDriverWait(browser, 10).until(
    lambda d: d.find_element(By.ID, "passesHeader").text != old_text
)

print('Website has successfully loaded.')

In [ ]:
driver.quit()

## Concluding Remarks

- **Selenium** is powerful for automating and scraping **dynamic** or **JavaScript-heavy** pages.  
- **Locating elements** can be done via ID, name, class, tag, link text, partial link text, XPath, or CSS.  
- **Mouse** and **keyboard** actions can simulate real user behavior.  
- Combine Selenium with **WebDriverWait** for reliability on sites with asynchronous loading.  
- Don’t forget best practices like **closing** the browser (`browser.quit()`) and being mindful about rate-limiting or server load.

For more advanced examples or a comprehensive PDF, refer to the original blog or advanced Selenium documentation.